In [1]:
from IPython.display import display
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

from xgboost import XGBRegressor

## Reading csv files

In [2]:
train_df = pd.read_csv("/kaggle/input/playground-series-s5e4/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s5e4/test.csv")
sample_submission_df = pd.read_csv("/kaggle/input/playground-series-s5e4/sample_submission.csv")

## Details of the data tables

In [3]:
def df_details(df):
    print("------------------------------------------------")
    print("df shape :", df.shape)
    print("------------------------------------------------")

    print("\n------------------------------------------------")
    print("df top 5 data :")
    print("------------------------------------------------")
    display(df.head(5))


    print("\n\n------------------------------------------------")
    print("df info :")
    print("------------------------------------------------")
    display(df.info())


    print("\n\n------------------------------------------------")
    print("df describe numeric data :")
    print("------------------------------------------------")
    display(df.describe())

    obs_cols = df.select_dtypes(include='object').columns
    if len(obs_cols) > 0:
        print("\n\n------------------------------------------------")
        print("df describe object data :")
        print("------------------------------------------------")
        display(df.describe(include=object))
    else:
        print("\n\n------------------------------------------------")
        print("No object data available :")
        print("------------------------------------------------")

    
    print("\n\n------------------------------------------------")
    print("Missing Values :")
    print("------------------------------------------------")
    print( df.isnull().sum()[df.isnull().sum() > 0] )
    
    
    missing_percentage = (df.isnull().sum() / len(df)) * 100 
    print("\n\n------------------------------------------------")
    print("Percentage of Missing values: (%) ")
    print("------------------------------------------------")
    print(missing_percentage[missing_percentage > 0])
    

    
    total_missing_percentage = (df.isnull().sum().sum() / (df.size)) * 100
    print("\n\n------------------------------------------------")
    print(f"Total missing values percentage: {total_missing_percentage:.2f}%")
    print("------------------------------------------------")

In [4]:
print("\n****************************************************")
print("Details of train_df: ")
print("\n****************************************************")
df_details(train_df)

print("\n\n\n\n****************************************************")
print("Details of test_df: ")
print("\n****************************************************")
df_details(test_df)


****************************************************
Details of train_df: 

****************************************************
------------------------------------------------
df shape : (750000, 12)
------------------------------------------------

------------------------------------------------
df top 5 data :
------------------------------------------------


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031




------------------------------------------------
df info :
------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           750000 non-null  int64  
 1   Podcast_Name                 750000 non-null  object 
 2   Episode_Title                750000 non-null  object 
 3   Episode_Length_minutes       662907 non-null  float64
 4   Genre                        750000 non-null  object 
 5   Host_Popularity_percentage   750000 non-null  float64
 6   Publication_Day              750000 non-null  object 
 7   Publication_Time             750000 non-null  object 
 8   Guest_Popularity_percentage  603970 non-null  float64
 9   Number_of_Ads                749999 non-null  float64
 10  Episode_Sentiment            750000 non-null  object 
 11  Listeni

None

...



------------------------------------------------
df describe numeric data :
------------------------------------------------


,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads
count,250000.000000,2.212640e+05,250000.000000,201168.000000,250000.000000
mean,874999.500000,4.192987e+02,59.716491,52.192796,1.355852
std,72168.927986,1.668545e+05,22.880028,28.445034,4.274399
min,750000.000000,2.470000e+00,2.490000,0.000000,0.000000
25%,812499.750000,3.578000e+01,39.250000,28.320000,0.000000
50%,874999.500000,6.397000e+01,59.900000,53.360000,1.000000
75%,937499.250000,9.415000e+01,79.390000,76.560000,2.000000
max,999999.000000,7.848626e+07,117.760000,116.820000,2063.000000




------------------------------------------------
df describe object data :
------------------------------------------------


,Podcast_Name,Episode_Title,Genre,Publication_Day,Publication_Time,Episode_Sentiment
count,250000,250000,250000,250000,250000,250000
unique,48,100,10,7,4,3
top,Tech Talks,Episode 71,Sports,Sunday,Night,Neutral
freq,7553,3492,28903,38906,65440,83671




------------------------------------------------
Missing Values :
------------------------------------------------
Episode_Length_minutes         28736
Guest_Popularity_percentage    48832
dtype: int64


------------------------------------------------
Percentage of Missing values: (%) 
------------------------------------------------
Episode_Length_minutes         11.4944
Guest_Popularity_percentage    19.5328
dtype: float64


------------------------------------------------
Total missing values percentage: 2.82%
------------------------------------------------


## Handling missing values and objects fields

### One Hot encoding

In [5]:
def encoding(X_train, X_valid, test):
    cat_cols = X_train.select_dtypes(include='object').columns.tolist()
    print("\n------------------------------------------------")
    print(f"Categorical columns: {cat_cols}")
    print("------------------------------------------------")

    # Fill missing values
    for df in [X_train, X_valid, test]:
        df[cat_cols] = df[cat_cols].fillna('missing')

    # Split columns by number of unique values in X_train
    onehot_cols = [col for col in cat_cols if X_train[col].nunique() <= 10]
    ordinal_cols = [col for col in cat_cols if X_train[col].nunique() > 10]

    # Encoded datasets, dropiong the categorical columns as we will encode it
    X_train_encoded = X_train.drop(cat_cols, axis=1).reset_index(drop=True)
    X_valid_encoded = X_valid.drop(cat_cols, axis=1).reset_index(drop=True)
    test_encoded = test.drop(cat_cols, axis=1).reset_index(drop=True)

    # One-Hot Encoding for categorical cols that have less than 11 unique values
    if onehot_cols:
        onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        onehot_encoder.fit(X_train[onehot_cols])

        cols = onehot_encoder.get_feature_names_out(onehot_cols)

        X_train_1hot = pd.DataFrame(onehot_encoder.transform(X_train[onehot_cols]), columns=cols)
        X_valid_1hot = pd.DataFrame(onehot_encoder.transform(X_valid[onehot_cols]), columns=cols)
        test_1hot = pd.DataFrame(onehot_encoder.transform(test[onehot_cols]), columns=cols)

        X_train_encoded = pd.concat([X_train_encoded, X_train_1hot], axis=1)
        X_valid_encoded = pd.concat([X_valid_encoded, X_valid_1hot], axis=1)
        test_encoded = pd.concat([test_encoded, test_1hot], axis=1)

    # Ordinal Encoding for categorical cols that have more than 10 unique values
    if ordinal_cols:
        ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        ordinal_encoder.fit(X_train[ordinal_cols])

        X_train_ord = pd.DataFrame(ordinal_encoder.transform(X_train[ordinal_cols]), columns=ordinal_cols)
        X_valid_ord = pd.DataFrame(ordinal_encoder.transform(X_valid[ordinal_cols]), columns=ordinal_cols)
        test_ord = pd.DataFrame(ordinal_encoder.transform(test[ordinal_cols]), columns=ordinal_cols)

        X_train_encoded = pd.concat([X_train_encoded, X_train_ord], axis=1)
        X_valid_encoded = pd.concat([X_valid_encoded, X_valid_ord], axis=1)
        test_encoded = pd.concat([test_encoded, test_ord], axis=1)

    return X_train_encoded, X_valid_encoded, test_encoded

### Imputation

In [6]:
def imputation(X_train, X_valid, test, strategy='mean'):
    my_imputer = SimpleImputer(strategy=strategy)
    
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
    imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
    imputed_test = pd.DataFrame(my_imputer.transform(test))
    
    imputed_X_train.columns = X_train.columns
    imputed_X_valid.columns = X_valid.columns
    imputed_test.columns = test.columns
    
    return imputed_X_train, imputed_X_valid, imputed_test

### Feature Scaling

In [7]:
def scaling(X_train, X_valid, test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)
    test_scaled = scaler.transform(test)

    X_train_data = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_valid_data = pd.DataFrame(X_valid_scaled, columns=X_valid.columns)
    test_data = pd.DataFrame(test_scaled, columns=test.columns)
    
    return X_train_data, X_valid_data, test_data

### Spliting dataset

In [8]:
y = train_df.Listening_Time_minutes
X = train_df.drop(['Listening_Time_minutes', 'id'], axis=1)
test_df = test_df.drop(['id'], axis=1)

print("X shape : ", X.shape)
print("y shape : ",y.shape)

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)
print("X_train shape : ",X_train.shape)
print("y_train shape : ",y_train.shape)
print("X_valid shape : ",X_valid.shape)
print("y_valid shape : ",y_valid.shape)

X shape :  (750000, 10)
y shape :  (750000,)
X_train shape :  (600000, 10)
y_train shape :  (600000,)
X_valid shape :  (150000, 10)
y_valid shape :  (150000,)


In [9]:
print("Categorical Encoding...")
X_train , X_valid, test_df = encoding(X_train, X_valid, test_df)
print("Numerical imputation...")
X_train , X_valid, test_df = imputation(X_train, X_valid, test_df)
print("Feature Scaling...")
X_train , X_valid, test_df = scaling(X_train, X_valid, test_df)

print("X_train shape: ", X_train.shape)
print("X_valid shape: ", X_valid.shape)
print("test_df shape: ", test_df.shape)

Categorical Encoding...

------------------------------------------------
Categorical columns: ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
------------------------------------------------
Numerical imputation...
Feature Scaling...
X_train shape:  (600000, 30)
X_valid shape:  (150000, 30)
test_df shape:  (250000, 30)


In [10]:
print("\n****************************************************")
print("Details of X_train: ")
print("\n****************************************************")
df_details(X_train)

print("\n****************************************************")
print("Details of X_valid: ")
print("\n****************************************************")
df_details(X_valid)

print("\n****************************************************")
print("Details of test_df: ")
print("\n****************************************************")
df_details(test_df)


****************************************************
Details of X_train: 

****************************************************
------------------------------------------------
df shape : (600000, 30)
------------------------------------------------

------------------------------------------------
df top 5 data :
------------------------------------------------


,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Genre_Business,Genre_Comedy,Genre_Education,Genre_Health,Genre_Lifestyle,Genre_Music,...,Publication_Day_Wednesday,Publication_Time_Afternoon,Publication_Time_Evening,Publication_Time_Morning,Publication_Time_Night,Episode_Sentiment_Negative,Episode_Sentiment_Neutral,Episode_Sentiment_Positive,Podcast_Name,Episode_Title
0,0.098693,0.019398,0.000000,-0.302721,-0.347041,-0.349034,-0.264859,3.083485,-0.351484,-0.302454,...,-0.409669,-0.561217,-0.593732,-0.556918,1.673548,1.414155,-0.709996,-0.704192,-0.673329,0.984346
1,-0.984701,-0.051004,-0.175800,1.434609,-0.347041,2.865053,-0.264859,-0.324308,-0.351484,-0.302454,...,-0.409669,-0.561217,-0.593732,1.795595,-0.597533,1.414155,-0.709996,-0.704192,-0.107437,0.522993
2,-0.625182,-1.329178,0.000000,1.434609,-0.347041,-0.349034,-0.264859,-0.324308,-0.351484,-0.302454,...,-0.409669,-0.561217,1.684261,-0.556918,-0.597533,-0.707136,-0.709996,1.420067,1.095084,1.481189
3,0.617819,0.174196,0.000000,1.434609,-0.347041,-0.349034,-0.264859,-0.324308,-0.351484,-0.302454,...,-0.409669,-0.561217,1.684261,-0.556918,-0.597533,1.414155,-0.709996,-0.704192,1.448767,-1.180468
4,1.121469,-1.491409,-0.948581,-1.171386,-0.347041,-0.349034,-0.264859,-0.324308,2.845082,-0.302454,...,-0.409669,-0.561217,-0.593732,1.795595,-0.597533,-0.707136,1.408458,-0.704192,0.316982,-0.577159




------------------------------------------------
df info :
------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Episode_Length_minutes       600000 non-null  float64
 1   Host_Popularity_percentage   600000 non-null  float64
 2   Guest_Popularity_percentage  600000 non-null  float64
 3   Number_of_Ads                600000 non-null  float64
 4   Genre_Business               600000 non-null  float64
 5   Genre_Comedy                 600000 non-null  float64
 6   Genre_Education              600000 non-null  float64
 7   Genre_Health                 600000 non-null  float64
 8   Genre_Lifestyle              600000 non-null  float64
 9   Genre_Music                  600000 non-null  float64
 10  Genre_News                   600000 non-null  float64
 11  Genre_S

None



------------------------------------------------
df describe numeric data :
------------------------------------------------


...



------------------------------------------------
df info :
------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Episode_Length_minutes       250000 non-null  float64
 1   Host_Popularity_percentage   250000 non-null  float64
 2   Guest_Popularity_percentage  250000 non-null  float64
 3   Number_of_Ads                250000 non-null  float64
 4   Genre_Business               250000 non-null  float64
 5   Genre_Comedy                 250000 non-null  float64
 6   Genre_Education              250000 non-null  float64
 7   Genre_Health                 250000 non-null  float64
 8   Genre_Lifestyle              250000 non-null  float64
 9   Genre_Music                  250000 non-null  float64
 10  Genre_News                   250000 non-null  float64
 11  Genre_S

None



------------------------------------------------
df describe numeric data :
------------------------------------------------


,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Genre_Business,Genre_Comedy,Genre_Education,Genre_Health,Genre_Lifestyle,Genre_Music,...,Publication_Day_Wednesday,Publication_Time_Afternoon,Publication_Time_Evening,Publication_Time_Morning,Publication_Time_Night,Episode_Sentiment_Negative,Episode_Sentiment_Neutral,Episode_Sentiment_Positive,Podcast_Name,Episode_Title
count,2.500000e+05,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,1.012400e+01,-0.006992,-0.001122,0.006396,-0.002362,0.001893,0.000583,0.001831,0.002427,0.001459,...,-0.002320,-0.001086,0.000797,0.003432,-0.003055,0.001460,-0.000984,-0.000476,-0.002084,-0.001903
std,5.061399e+03,1.000501,0.999413,3.713020,0.997001,1.002379,1.001025,1.002523,1.003021,1.002190,...,0.997640,0.999339,1.000436,1.002119,0.998353,1.000517,0.999658,0.999832,0.998148,1.000499
min,-2.001351e+00,-2.509401,-2.045671,-1.171386,-0.347041,-0.349034,-0.264859,-0.324308,-0.351484,-0.302454,...,-0.409669,-0.561217,-0.593732,-0.556918,-0.597533,-0.707136,-0.709996,-0.704192,-1.663640,-1.783777
25%,-8.115517e-01,-0.901954,-0.692424,-1.171386,-0.347041,-0.349034,-0.264859,-0.324308,-0.351484,-0.302454,...,-0.409669,-0.561217,-0.593732,-0.556918,-0.597533,-0.707136,-0.709996,-0.704192,-0.885539,-0.861069
50%,0.000000e+00,0.001033,0.000000,-0.302721,-0.347041,-0.349034,-0.264859,-0.324308,-0.351484,-0.302454,...,-0.409669,-0.561217,-0.593732,-0.556918,-0.597533,-0.707136,-0.709996,-0.704192,-0.036700,0.026150
75%,8.374001e-01,0.853294,0.734851,0.565944,-0.347041,-0.349034,-0.264859,-0.324308,-0.351484,-0.302454,...,-0.409669,-0.561217,1.684261,-0.556918,1.673548,1.414155,1.408458,1.420067,0.882875,0.842391
max,2.530699e+06,2.531144,2.529911,1790.884559,2.881501,2.865053,3.775598,3.083485,2.845082,3.306284,...,2.440995,1.781841,1.684261,1.795595,1.673548,1.414155,1.408458,1.420067,1.660976,1.729611




------------------------------------------------
No object data available :
------------------------------------------------


------------------------------------------------
Missing Values :
------------------------------------------------
Series([], dtype: int64)


------------------------------------------------
Percentage of Missing values: (%) 
------------------------------------------------
Series([], dtype: float64)


------------------------------------------------
Total missing values percentage: 0.00%
------------------------------------------------


### XGBoost

In [11]:
model = XGBRegressor(
    n_estimators=50000, 
    learning_rate=0.01,
    early_stopping_rounds=10
)

model.fit(X_train, y_train, 
         eval_set=[(X_valid, y_valid)], 
         verbose=False
    )

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=10,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [12]:
y_pred = model.predict(X_valid)
mae = mean_absolute_error(y_valid, y_pred)
mse = mean_squared_error(y_valid, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")

RMSE: 12.9792
MSE: 168.4593
MAE: 9.4205


In [13]:
final_prediction = model.predict(test_df).flatten()
submission = pd.DataFrame({'id': sample_submission_df['id'], 'Listening_Time_minutes': final_prediction})
submission.to_csv('submission.csv', index=False)